In [ ]:
import numpy as np
import pandas as pd
import torch
import warnings
warnings.filterwarnings('ignore')

# Dataset


In [ ]:
import pickle
all_tables = pickle.load(open('data/Corona/all_tables.pkl','rb')) 
all_claims = pickle.load(open('data/Corona/all_claims.pkl','rb')) 
user_claims = pickle.load(open('data/Corona/user_claims.pkl','rb')) 

In [ ]:
from tqdm import tqdm

table_columns = {1:'table',2:'country',3:'january',4:'february',5:'march',6:'april',7:'may',8:'june'}
G=nx.Graph()
K = 3

i = 0
nodes_labels = {}
row_ids = {}
id_rows = {}

for row in tqdm(all_tables):
    i+=1
    row_name = str('RW'+str(i))
    G.add_node(row_name , label= row_name, type='Row')
    row_ids[row_name] = ' '.join([r for r in row])
    id_rows[' '.join(row)] = row_name
    
    j=0
    for cl in row:
        j+=1
        col_name = table_columns[j]
        if cl == '': continue
        if not G.has_node(col_name):     G.add_node(col_name , label= col_name, type='Column')
        n_grams = [gr.replace(' ','_') for gr in find_all_n_grams(str(cl),K)]
        for tg in n_grams:
            G.add_node(tg,label=tg, type='Token')
            G.add_edge(row_name,tg)
            G.add_edge(col_name,tg)
            


i = 0
claim_ids = {}
id_claim = {}
all_claims.update(user_claims)


for claim in tqdm(all_claims):
    i += 1
    text = (claim)
    claim_name = str('Claim'+str(i))
    G.add_node(claim_name , label= claim_name, type='Claim')
    claim_ids[claim_name] = claim
    id_claim[claim] = claim_name
    
    n_grams = [gr.replace(' ','_') for gr in find_all_n_grams(text,K)]

    for tg in n_grams:
        if not G.has_node(tg):            
            continue
           
        if not G.has_edge(claim_name,tg):            G.add_edge(claim_name,tg)
        

In [ ]:
ground_truth = {}

for cl in claim_ids:
    ground_truth[cl] = []
    if claim_ids[cl] in user_claims:
        for r in user_claims[claim_ids[cl]]:
            for rr in id_rows:
                if ' '.join(reversed(r)) in rr:             
                    ground_truth[cl].append(id_rows[rr])
    else:
        for r in id_rows:
            if ' '.join(all_claims[claim_ids[cl]][0:2]) in r:
                ground_truth[cl].append(id_rows[r])

# Method0: BM25

In [ ]:
from gensim.summarization.bm25 import get_bm25_weights
from gensim.summarization.bm25 import BM25

In [ ]:
corpus = [row for row in all_tables]
results = BM25(corpus)

In [ ]:
import numpy as np
query_BM25 = {}
for query in tqdm(ground_truth):
    text = claim_ids[query]
    scores = results.get_scores(text.split())
    arr = np.array(scores)
    topK = arr.argsort()[::-1]
    query_BM25[query] = [(corpus[idx],scores[idx]) for idx in topK]

In [ ]:
for KK in [1,2,3,5,10,20,50,50000]: 
    i = 0
    precision,recall,fs = 0,0,0
    MAP, MRR, hasP = 0,0,0

    for query in query_BM25:
        if query not in ground_truth or len(ground_truth[query])==0: continue
        if query in [id_claim[c] for c in user_claims]: continue
            
        i+=1
        preds = [id_rows[' '.join(f)] for (f,j) in query_BM25[query]][0:KK]
        golds = [g for g in ground_truth[query]]
        
        MAP += utils.MAP_K(golds,preds)
        MRR += utils.MRR(golds,preds)
        hasP += utils.HAS_POSITIVE(golds,preds)
        
    print('\n#################### ' + str(KK) + ' ###########################\n')
    print('MRR:',MRR/i,'MAP:',MAP/i, 'HAS POSITIVE:', hasP/i)


# Method1: Unsupervised SentenceBERT

In [ ]:
from gensim.parsing.preprocessing import remove_stopwords

vocabs = set()
for node in G.nodes():
    if G.nodes()[node]['type'] == 'Token':
        if len(node.split('_')) == 1: vocabs.add(node)

def return_filtered(text):
    text = remove_stopwords(normalize_text(text))
    t = ''
    for token in word_tokenize(text):
        if token in vocabs: t += token + ' '
    return t   

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer('bert-large-nli-stsb-mean-tokens')

In [ ]:
special_tokens_dict = {'additional_special_tokens': ['[COL]','[VAL]']}

In [ ]:
word_embedding_model = model._first_module()   #Your models.Transformer object
word_embedding_model.tokenizer.add_special_tokens(special_tokens_dict)
word_embedding_model.auto_model.resize_token_embeddings(len(word_embedding_model.tokenizer))

In [ ]:
table_content = []
for t in all_tables:
  for c in range(0,len(t)):
    text += ' [COL] ' + str(table_columns[c+1]) + ' [VAL] ' + str(t[c])
  
  table_content.append(text)

In [ ]:
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

claim_embs = model.encode([t for t in table_content],show_progress_bar=True)

In [ ]:
pred_quey = {}

for claim in tqdm([g for g in all_claims.keys()],position=0):
  text = claim 

  m_emb = model.encode(claim)
  
  temp = []
  for rv in range(0,len(row_ids)):
    temp.append(([r for r in row_ids.keys()][rv],cosine_similarity(m_emb.reshape(1, -1),claim_embs[rv].reshape(1, -1))[0][0]))
  pred_quey[id_claim[claim]] = sorted(temp,key=lambda dist:dist[1],reverse=True)
  

In [ ]:
for KK in [1,3,5,10,20,50,500]: 
    i = 0
    MAP, MR, hasP = 0,0,0

    for query in pred_reviews:
        if query not in ground_truth or len(ground_truth[query])==0: continue
        if query not in [id_claim[c] for c in user_claims]: continue
            
        i+=1
        preds = [f for (f,j) in pred_query[query]][0:KK]
        golds = [g for g in ground_truth[query]]
        
        MAP += MAP_K(golds,preds)
        MR += MRR(golds,preds)
        hasP += HAS_POSITIVE(golds,preds)
        
    print('\n#################### ' + str(KK) + ' ###########################\n')
    try:
        print('MRR:',MR/i,'MAP:',MAP/i, 'HAS POSITIVE:', hasP/i)
    except:
        print('y')

# Method2: Supervised SentenceBERT

In [ ]:
from sentence_transformers import SentenceTransformer
SBmodel = SentenceTransformer('bert-large-nli-stsb-mean-tokens')

In [ ]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
table_embs = {}
for row in tqdm(id_rows,position=0):
  table_embs[row] = model.encode(row)


In [ ]:
query_score = []

for claim in tqdm([g for g in all_claims.keys()],position=0):
  text = claim
    

  m_emb = model.encode(text)
    
  for rw in id_rows:
    temp = []
    temp.append(cosine_similarity(m_emb.reshape(1, -1),table_embs[rw].reshape(1, -1))[0][0])
      
    if id_rows[rw] in ground_truth[claim]: temp.append(1)
    else: temp.append(0)
    query_score.append(temp)

In [ ]:
import numpy as np

dataset = np.array(query_score)
X = dataset[:,0:1]
y = dataset[:,1]

In [ ]:
import keras
from keras import losses,optimizers

model = Sequential()
model.add(Dense(20, input_dim=1, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam(lr=1e-3), metrics=['accuracy'])

In [ ]:
class_weight = {0: 1.,1: 500.}

model.fit(X, y, epochs=50, batch_size=2048,class_weight=class_weight)

In [ ]:
query_predictions = {}

for query in tqdm(ground_truth,position=0):
  if query not in [id_claim[c] for c in user_claims]: continue

    
  text = claim_ids[query]
            
  m_emb = SBmodel.encode(text)

  seen = []
  data,scores = [],[]

  for rw in id_rows:
    seen.append(rw)
    data.append(cosine_similarity(m_emb.reshape(1, -1),table_embs[rw].reshape(1, -1))[0][0])

  res = model.predict(np.array(data))
        
  for i in range(0,len(res)):
      scores.append((seen[i],res[i][0]))
        
  query_predictions[query] = sorted(scores, key=lambda dist: dist[1],reverse = True)

In [ ]:
for KK in [1,2,3,5,10,20,50,500]: 
    i = 0
    precision,recall,fs = 0,0,0
    MAP, MR, hasP = 0,0,0

    for query in query_predictions:
        if id_claim[query] not in ground_truth or len(ground_truth[id_claim[query]])==0: continue
        if query in [id_claim[c] for c in user_claims]: continue
        
        i+=1
        preds = [id_rows[f] for (f,j) in query_predictions[query]][0:KK]
        golds = [f for f in ground_truth[id_claim[query]]]

        MAP += MAP_K(golds,preds)
        MR += MRR(golds,preds)
        hasP += HAS_POSITIVE(golds,preds)
        
    print('\n#################### ' + str(KK) + ' ###########################\n')
    print('MRR:',MR/i,'MAP:',MAP/i, 'HAS POSITIVE:', hasP/i)

# Method3: Reranking

In [ ]:
corona_SB = query_predictions
corona_BM25 = query_BM25 

In [ ]:
import dlib
data = dlib.ranking_pair()

In [ ]:
import time
st = time.time()
for query in tqdm(ground_truth,position=0):

    m_BM = [' '.join(i) for (i,j) in corona_BM25[ query]]
    m_SB = [i for (i,j) in corona_SB[query]]
    
    
    for r in row_ids:
        if r in ground_truth[query]: 
            data.relevant.append(dlib.vector([m_BM.index(row_ids[r])+1, m_SB.index(r)+1]))
        else:  data.nonrelevant.append(dlib.vector([m_BM.index(row_ids[r])+1, m_SB.index(r)+1]))

trainer = dlib.svm_rank_trainer()
trainer.c = 1000

rank = trainer.train(data)
print(time.time()-st)

In [ ]:
st = time.time()
i=0
rerank_corona = {}

for query in tqdm(ground_truth,position=0):
    i+=1

    m_BM = [' '.join(i) for (i,j) in corona_BM25[ query]]
    m_SB = [i for (i,j) in corona_SB[query]]

    temp = []
    for r in row_ids:
        temp.append((r,rank(dlib.vector([m_BM.index(row_ids[r])+1, m_SB.index(r)+1]))))
    temp = sorted(temp, key=lambda dist: dist[1],reverse = True)
    rerank_corona[query] = temp
print((time.time()-st)/i)

In [ ]:
for KK in [1,2,3,5,10,20,50,500]: 
    i = 0
    precision,recall,fs = 0,0,0
    MAP, MR, hasP = 0,0,0

    for query in rerank_corona:
        if query not in ground_truth or len(ground_truth[query])==0: continue
        if query in [id_claim[c] for c in user_claims]: continue
        
        i+=1
        preds = [f for (f,j) in rerank_corona[query]][0:KK]
        golds = [f for f in ground_truth[query]]

        MAP += MAP_K(golds,preds)
        MR += MRR(golds,preds)
        hasP += HAS_POSITIVE(golds,preds)
        
    print('\n#################### ' + str(KK) + ' ###########################\n')
    print('MRR:',MR/i,'MAP:',MAP/i, 'HAS POSITIVE:', hasP/i)

# Method4: Doc2Vec

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
data = []
for query in all_tables:
    text = ' '.join([str(m) for m in query]) 
    data.append(text)
tagged_data = [TaggedDocument(words=word_tokenize(_d), tags=[str(i)]) for i, _d in enumerate(data)]

In [ ]:
%env PYTHONHASHSEED=0
max_epochs = 10
vec_size = 50

model = Doc2Vec(size=vec_size, min_count=10, dm =0, workers=1, window=4,seed=0, epochs=max_epochs
                )

model.build_vocab(tagged_data)

model.train(tagged_data, total_examples=model.corpus_count,epochs=model.epochs)


print("Model Saved")

In [ ]:
import numpy as np
query_d2v = {}
for query in tqdm(ground_truth):
    text = claim_ids[query]
    
    query_d2v[query] = utils.cosine_distance(model,text,id_rows,500)

In [ ]:
for KK in [1,2,3,5,10,20,50,2000]: 
    i = 0
    precision,recall,fs = 0,0,0
    MAP, MRR, hasP = 0,0,0

    for query in query_d2v:
        if claim_ids[query] not in ground_truth: continue
        i+=1
        preds = [id_review[f] for (f,j) in query_d2v[query]][0:KK]
        golds = [f for f in ground_truth[row_ids[query]]]

        MAP += utils.MAP_K(golds,preds)
        MRR += utils.MRR(golds,preds)
        hasP += utils.HAS_POSITIVE(golds,preds)
        
    print('\n#################### ' + str(KK) + ' ###########################\n')
    print('MRR:',MRR/i,'MAP:',MAP/i, 'HAS POSITIVE:', hasP/i)